# El método de Euler aplicado a sistemas de EDOs

En este documento, por un lado se va a implementar el método de Euler, y por otro se aplicará dicho método para simular la evolución de dos sistemas de EDOs distintos.

<ul id="top">
<li><a href="#1-Lotka-Volterra-con-caza-periódica">1-Lotka-Volterra con caza periódica</a></li>
<li><a href="#2-Implementación-del-método-de-Euler">2-Implementación del método de Euler</a></li>
<li><a href="#3-Simulación-de-un-péndulo">3-Simulación de un péndulo</a></li>
<li><a href="#Valoración">Valoración</a></li>
</ul>  

In [ ]:
using Plots
using DelimitedFiles

## 1-Lotka-Volterra con caza periódica
### Variante del sistema presa-depredador de Lotka-Volterra con caza periódica

Consideramos la siguiente variante no autónoma del sistema
   presa-depredador de Lotka-Volterra:

<a id='Edo-LV'></a>
\begin{equation*}
\frac{d}{dt} v = \left(a-b w - g(t) \right)\, v, \quad \frac{d}{dt}w = \left(c v-d\right)\, w. \hspace{3em} \tag{1}
\end{equation*}

El tiempo $t$ se mide en meses.
La variable de estado $v$ representa el número de alces de un parque
natural protegido, y $w$ el número de lobos. Los símbolos $a,b,c,d$ se refieren a parámetros constantes del problema, que en nuestro caso fijamos como $a=1$, $b=1/50$, $c=3/100$, $d=1$.
Se supone que los guardas
del parque cazan de forma periódica (con un periodo de doce meses)
cierto porcentaje de alces, porcentaje que varía 
dependiendo de los meses del año, 
donde $g(t)$ es una función períodica de $t$ con un período de 12 meses, que
representa la tasa de caza de alces por unidad de tiempo. Como ejemplo ilustrativo, aquí consideraremos la función 

\begin{equation}
  g(t)= \frac{1}{20} \big(\sin\big(\frac{\pi\, t}{12}\big)\big)^2.
\end{equation}

Dicha función alcanza su valor máximo (igual a $1/20$) a mitad de año, cuando $t=6$, y se anula al principio y final del año (es decir, $t=0$ y $t=12$). 

En las transparencias de la tercera parte del Tema 1 se muestra que el sistema (1) se puede escribir de forma compacta de la forma

<a id='Edo-Compacta'></a>
\begin{equation}
\frac{d}{dt} u = f(t,u,p). \hspace{7em} \tag{2}
\end{equation}


### 1.1- Ejercicio

- **Implementar la función $f$ que hace que el sistema  [(1)](#Edo-LV) sea equivalente a [(2)](#Edo-Compacta).**

In [ ]:
function f(t,u,p)
    v=u[1]
    w=u[2]
    a = p[1]
    b = p[2]
    c = p[3]
    d = p[4]
    ?
    dv = ?
    dw = ?
    return [dv, dw]
end

### 1.2- Ejercicio

- Consideremos la solución $u(t)=(v(t),w(t))$ de [(1)](#Edo-LV) con los datos iniciales $v(0)=51$, $w(0)=33$. **Calcular la aproximación $u_1=(v_1,w_1)$ de $u(t_1)=(v(t_1),w(t_1))$ en $t_1 = 0.025$ aplicando un paso de longitud $h=0.025$ del método de Euler.**

In [ ]:
p = [1.,1/50,3/100,1.]
u0 = [51., 33.]
t0 = 0.
h = ?
t1 = ?
u1 = ?

> **Comprobación**
>
>El resultado debería coincidir con el mostrado en el documento 
 <a href="https://egela1819.ehu.eus/mod/resource/view.php?id=1022048">Ilustración del método de Euler para sistemas de EDOs </a> (disponible en el aula virtual).

### 1.3- Ejercicio

- **Hacer otro tanto para calcular la aproximación $u_2=(v_2,w_2)$ de $u(t_2)=(v(t_2),w(t_2))$ en $t_2=2h$ (para $h=0.025$) obtenida por el método de Euler a partir de la aproximación $u_1=(v_1,w_1)$ de $u(t_1)=(v(t_1),w(t_1))$ previamente calculada.**

In [ ]:
t2 = ?
u2 = ?

<a href="#top">Back to the top</a>

## 2-Implementación del método de Euler

Queremos simular numéricamente la evolución de sistemas modelados por 
ecuaciones diferenciales ordinarias (EDOs) de dimensión $d\geq 1$ de la forma [(2)](#Edo-Compacta)
donde $u =(u^1,\ldots,u^d) \in \mathbb{R}^d$ es el _vector de estado_ del sistema, y $p \in \mathbb{R}^m$ es un vector de parámetros constantes del sistema.

Sabemos que, fijado el vector $p$ de parámetros constantes, 
dados $t_0 \in \mathbb{R}$ y $u_0 =(u^1_0,\ldots,u^d_0) \in \mathbb{R}^d$, existe una única solución $u(t)$ de [(2)](#Edo-Compacta) que satisfaga la condición inicial

<a id='Cond-Ini'></a>
\begin{equation}
  u(t_0)=u_0. \hspace{8em} \tag{3}
\end{equation}

En la práctica, nos interesará calcular de forma aproximada la solución $u(t)$ para un cierto intervalo temporal $t \in [t_0,T]$. Se aproximará dicha solución en una discretización del tiempo obtenida de dividir el intervalo temporal $[t_0,T]$ en $n$ partes iguales. Es decir,  se tomará los valores discretos del tiempo $t_0,t_1,t_2,\ldots,t_{n-1},t_n=T$, donde $t_k=t_{0}+k h$, con $h=(T-t_0)/n$, y se calcularán las aproximaciones 
$u_k =(u^1_k,\ldots,u^d_k) \approx u(t_k)$ para $k=0,1,\ldots,n$.

### 2.1 - Ejercicio

- **Para realizar los ejercicios de simulación que siguen, se debe primeramente implementar en Julia el método de Euler. Se definirá una función (que llamaremos Euler) que toma como argumentos de entrada $u0, t0, T, n, f, p$.**
     - El argumento de entrada $f$ es el nombre de una función previamente definida, tal que dados $t \in \mathbb{R}$, un vector $u \in \mathbb{R}^d$, y un vector de parámetros $p\in \mathbb{R}^m$, devuelve un vector $f(t,u,p) \in \mathbb{R}^d$. Dicha función determina un sistema de EDOs de la forma (2).
     - El significado del resto de los argumentos de entrada de dicha función se puede deducir de las consideraciones de arriba. 
     - La función Euler debe devolver como argumento de salida una matriz $W \in \mathbb{R}^{(n+1) \times (d+1)}$
 \begin{eqnarray*}
W = \left(
     \begin{array}{cccc}
t_0    &   u^1_0 & \cdots& u^d_0  \\
t_1    &    u^1_1 & \cdots& u^d_1  \\
\vdots &  \vdots & \ddots& \vdots \\
t_{n}  &      u^1_n & \cdots& u^d_n
     \end{array}
\right),
\end{eqnarray*} 
donde $t_k = t_0 + k \, (T-t_0)/n$, y los vectores $u_k  = (u^1_k,\ldots,u^d_k) \in \mathbb{R}^d$ ($k=1\ldots,n$) son aproximaciones de $u(t_k)$ que se han obtenido aplicando el método de Euler al sistema [(2)](#Edo-Compacta) con la condición inicial [(3)](#Cond-Ini) utilizando una longitud de paso  $h=(T-t_0)/n$.

In [ ]:
function  Euler(u0, t0, T, n, f, p)
    h = ?   # Calculo de la longitud de paso
    # Se deduce la dimension del sistema de EDOs a partir del numero 
    # de componentes del vector de estado inicial
      d = length(u0)  
    # Inicializacion a cero de la matriz de salida de resultados
      W = zeros(n+1,d+1)
      W[1,:] = [t0; u0]  # Se rellena la primera fila de la matriz de resultados 
      tj = t0
      uj = u0
    # Calculo sucesivo de los tiempos tj y las aproximaciones uj de u(tj)
    for j in 1:n
        duj = ?
        uj = ?
        tj = ?
        W[j+1,:] = [tj; uj] # Se introduce el resultado del paso j-ésimo 
                            # en la fila (j+1)-ésima de la matriz de resultados
    end
    return W
  end

### 2.2 - Ejercicio

Una vez implementada la función Euler,
 - **Resolver numéricamente (con el método de Euler) el sistema con valores iniciales $v(0)=51$, $w(0)=33$, para $t\in[0,30]$. Para ello, considerar $n=1200$, y por tanto $h=30/1200=0.025$.**

In [ ]:
?
?
?
res = Euler(?,?,?,?,?,?)

> **Comprobación**
>
>Los resultados en los primeros tiempos deberían coincidir con los mostrados en el documento 
 <a href="https://egela1819.ehu.eus/mod/resource/view.php?id=1022048">Ilustración del método de Euler para sistemas de EDOs </a> (disponible en el aula virtual). 

 - **Obtener una figura con las gráficas de ambas componentes de la solución con respecto del tiempo.**

In [ ]:
tt = res[:,1]  #Se extrae la primera columna de la tabla res
vv = res[:,2]  #Segunda columna de la tabla de resultados res
ww = res[:,3]  #Tercera columna de la tabla res

plot(tt,[vv, ww],title="Evolución de alces y lobos",
      xlabel="t",label=["alces (h=0.025)" "lobos (h=0.025)"])

### 2.3- Ejercicio

Una opción para comprobar si los resultados de la evolución de las poblaciones de las dos especies son fiables (en el sentido de que dan aproximaciones fiables de la solución de nuestro modelo matemático), consiste en volver a aplicar el método de Euler, pero con una longitud de paso $h$ más pequeña que la utilizada arriba ($h=30/1200=0.025$). Por ejemplo, con $h= 0.025/2 = 0.0125$.  Eso se podría hacer fácilmente con la función Euler implementada en el Ejercicio 2.1, llamando a la función con $n=2400$ en lugar de con $n=1200$, pero en dicho caso, la tabla de resultados será mayor, pues se guardarán los resultados calculados para una discretización más fina.

En lugar de ello, implementaremos una nueva versión de la función Euler, que toma un argumento de entrada $m$ adicional (que debe ser un número entero positivo). 

- En caso de que se le llame a la nueva función Euler con $m=1$, procederá de la misma forma que la función Euler implementada en el ejercidio 2.1, obteniendo aproximaciones $u_k \approx u(t_k)$ ($k=1,2,\ldots,n$)  para $t_k = t_0 + k \, (T-t_0)/n$, aplicando el método de Euler al sistema [(2)](#Edo-Compacta) con la condición inicial [(3)](#Cond-Ini).  

- En caso de que $m>1$, se obtienen aproximaciones $u_k\approx u(t_k)$ para los mismos tiempos $t_k$, calculando $u_k$ como resultado de la aplicación de $m$ pasos de longitud $h= (T-t0)/(n\, m)$ del método de Euler a partir de $u_{k-1}$. 

- El argumento de entrada $m$ será opcional, con valor por defecto $m=1$. (Es decir, si no se suministra dicho argumento de entrada, se considera que $m=1$.)

- **Implementar dicha nueva versión de la función Euler**

In [ ]:
function  Euler(u0, t0, T, n, f, p, m=1)
    h = (T-t0)/(m*n);   # Calculo de la longitud de paso
    # Se deduce la dimension del sistema de EDOs a partir del numero 
    # de componentes del vector de estado inicial
      d = length(u0)  
    # Inicializacion a cero de la matriz de salida de resultados
      W = zeros(n+1,d+1)
      W[1,:] = [t0; u0]  # Se rellena la primera fila de la matriz de resultados 
      tj = t0
      uj = u0
    # Calculo sucesivo de los tiempos tj y las aproximaciones uj de u(tj)
    for j in 1:n
      for i in 1:m  
        ?
        ?
        ?
      end      
        W[j+1,:] = ?    # Se introduce el resultado del paso j-ésimo 
                        # en la fila (j+1)-ésima de la matriz de resultados
    end
    return W
  end


 - **Resolver numéricamente (con el método de Euler) el sistema con valores iniciales $v(0)=51$, $w(0)=33$, para $t\in[0,30]$, obteniendo aproximaciones $(v_k, w_k) \approx (v(t_k), w(t_k))$ ($k=1,2,\ldots,n$)  para $t_k = t_0 + k \, (T-t_0)/n$ con $n=1200$, pero a diferencia de en el Ejercicio 2.2, esta vez utilizando como longitud de paso $h=30/2400=0.0125$.**

In [ ]:
?
?
?
res2 = Euler(?,?,?,?,?,?,?)

 - **Añadir a la figura obtenida en el Ejercicio 2.2,  las gráficas de ambas componentes de la solución con respecto del tiempo correspondientes a los resultados obtenidos con $h=0.0125$.**

In [ ]:
tt2 = res2[:,1]
vv2 = res2[:,2]
ww2 = res2[:,3]

plot!(tt2,[vv2, ww2],title="Evolución de alces y lobos",
      xlabel="t",label=["alces (h=0.0125)" "lobos (h=0.0125)"])

 - **¿Cual de los resultados es de esperar que sea más precisos, los obtenidos con $h=0.0125$ o los obtenidos con $h=0.025$?**

> **Respuesta**
>
>??

 - **Al comparar los resultados obtenidos con $h=0.0125$ con los obtenidos con $h=0.025$, ¿se puede concluir que los resultados obtenidos con $h=0.025$ son muy precisos?.**
 
> **Respuesta**
>
>??

<a href="#top">Back to the top</a>

## 3-Simulación de un péndulo
**Simulación de un péndulo bajo el efecto de la gravedad y la resistencia del aire**

Consideraremos la siguiente EDO de segundo orden, que modela el
  movimiento de un pendulo formado por una esfera de superficie suave unida al eje de giro por una varilla. Se supone que el péndulo está inmerso en aire, y se tiene en cuenta el efecto sobre la esfera de la acceleración de la gravedad, la resistencia del aire al movimiento de la esfera, pero no sobre la varilla, que se considera de masa y diametro de sección transversal despreciable (con respecto a la masa y el radio de la esfera respectivamente).  
  
  En dicho modelo, $\theta(t)$ es el ángulo (con respecto a la posición vertical) de la varilla en el instante $t$,
$L$ es la distancia del eje de giro al centro de la esfera, $g$ es la acceleración de la
gravedad, $R$ es el radio de la esfera, $\rho_e$ es la densidad de la esfera, $\rho_a$ es la densidad del aire, y $\mu_a$ es la densidad dinámica del aire. La ecuación diferencial (de segundo orden) que se obtiene es

<a id='Edo-Pendulo'></a>
\begin{equation*}
        \frac{d^2 \theta}{dt^2} = -\frac{g}{L}\, \sin(\theta) - \sigma\, \frac{3 \rho_a\, L\, C_D(Re)}{8\, \rho_e \, R}\, \left|\frac{d\theta}{dt}\right|^2, \tag{4}
\end{equation*}
    
donde  $\sigma=1$ si $\frac{d\theta}{dt}>0$, y $\sigma=-1$ en caso contrario, y
\begin{equation*}
        Re = \frac{2\, \rho_a\, R\, L}{\mu_a} \left| \frac{d \theta}{dt}\right|, \quad
        C_D(Re)  = \left\{
        \begin{matrix}
       \frac{24}{Re} + \frac{2}{5} +\frac{6}{1+\sqrt{Re}} & \mbox{ si} & Re>0,\\
       0  \phantom{ \frac{2}{5} +\frac{6}{1+\sqrt{Re}} }  & \mbox{ si} & Re=0.
        \end{matrix}
        \right. 
\end{equation*}

Aquí, $C_D(Re)$ es el _coeficiente de arrastre_ de un fluido cualquiera sobre una esfera de superficie suave (ya considerado anteriormente, que es función del _número de Reynols_ $Re$, directamente proporcional a la velocidad $|v|=L\, |d\theta/dt|$ de la esfera).

Consideraremos los siguientes valores de los parámetros del problema, correspondientes al caso de un péndulo de 20cm de longitud con una esfera de aluminio de 2cm de radio, inmersa en  aire (a $20^oC$  con una presión estándar de una atmósmera):   


<a id='Inicial-cond'></a>
\begin{equation}
L=0.2\, \mathrm{m}, \ 
R=0.02\, \mathrm{m}, \  
\rho_e=2712\, \mathrm{Kg}/\mathrm{m^3}, \ 
\rho_a = 1.205\,  kg/m^3, \ 
\mu_a = 1.789\times 10^{-5}\,  kg/(\mathrm{m} \,\mathrm{seg}), \ 
g=9.8\, \mathrm{m}/\mathrm{seg}^2. \tag{5}
\end{equation}

Queremos simular la evolución del ángulo $\theta(t)$ con respecto del tiempo, para $t \in [0,50]$.  En particular, consideraremos el caso en que inicialmente se deja la varilla inmóvil en posición casi vertical
 \begin{equation*}
   \theta(0)=\pi-0.001, \quad \frac{d\theta}{dt}(0)=0.
 \end{equation*}

### 3.1- Ejercicio

- **Reescribir la ecuación de segundo órden [(4)](#Edo-Pendulo) como un sistema de dos ecuaciones de primer orden, añadiendo para ello una nueva variable $\omega$ para la velocidad angular $\frac{d\theta}{dt}$, de tal modo que el vector de estados en este sistema es $u = (\theta, \omega)$. Implementar la función $f$ que hace que dicho sistema de dos ecuaciones de primer orden sea equivalente a [(2)](#Edo-Compacta).**

In [ ]:
function f(t,u,p)
    L = p[1]
    g = p[2]
    rhoe = p[3]
    R = p[4]
    rhoa = p[5]
    mu = p[6]
    theta = u[1]
    omega = u[2]
    if omega==0 
        Cd = 0
    else
        Re = ?
        Cd = ?
    end
    dtheta = ?
    aux = ?  # Variable auxiliar
    domega = ?       
    [dtheta,domega] 
end

- **Evaluar la función f recién implementada para los valores de los parámetros constantes dados 
en [(5)](#Inicial-cond) para $t=0$, $\theta=\pi/2$, $\omega=0.1$.**

In [ ]:
 L = ?
 g = ?
 rhoe = ?  # Densidad del aluminio
 R = ?
 rhoa = ? # densidad del aire a 20ºC al nivel del mar
 mu = ? # viscosidad dinámica del aire a 20ºC al nivel del mar
 p = [L, g, rhoe, R, rhoa, mu]
 f(?, ?, p)

> **Comprobación**
>
> Debería de dar como resultado el vector [0.1, -49.00002607219265]. 

### 3.2- Ejercicio

- **Aproximar la solucion $\theta(t)$ para $t=t_0,t_1,t_2,\ldots,t_n=50$, donde $t_k=50 \, k/n$, $n=24000$, utilizando el método de Euler con longitud de paso $h=50/n=1/480$.**

In [ ]:
  t0 = 0.
  T = 50.
  u0 = [?, ?]
  n = ?
  res = ?

- **Representar gráficamente la evolución de $\theta(t)$ con respecto del tiempo.**

In [ ]:
  tt = ?
  thetas = ?
  omegas = ?
  plot(tt,thetas,title="Resultado numérico de evolución del ángulo del péndulo", 
                 xlabel = "t", ylabel="\\theta", label="Euler, h=1/480")

### 3.3- Ejercicio

- **Guardar la secuencia de valores de los ángulos obtenidos en un fichero, y probar a utilizarlo para realizar una animación del movimiento del péndulo. Interpretar los resultados para concluir si la simulación del movimiento del péndulo tiene sentido desde el punto de vista físico.**

In [ ]:
data="./angulos_pendulo.txt"
f1=open(data,"w")
writedlm(f1,thetas)
close(f1)

- **Realizar una animación del movimiento del péndulo ejecutando el codigo JavaScripts: <a href="http://www.ehu.eus/ccwmuura/irakaskuntza/sinum/animaciones/pendulo/pendulo_desde_fichero.html">Fichero html</a>, especificando el nombre del archivo en el que han guardado la secuencia de valores de los ángulos.**

- **¿Parece un movimiento realista?**

> **Respuesta**
>
>???

### 3.4- Ejercicio

- **Volver a aplicar el método de Euler para obtener aproximaciones de la solución para $t=t_0,t_1,t_2,\ldots,t_{24000}$, donde $t_k=\frac{50 \, k}{24000}$, pero esta vez con longitud de paso $h=1/960$.**

In [ ]:
res2 = ?

 - **Representar en una misma figura la evolución del ángulo del péndulo correspondiente a la aplicación de Euler con $h=1/480$ y con $h=1/960$ respectivamente.**

In [ ]:
tt2 = ?
thetas2 = ?
omegas2 = ?
plot(tt,thetas,title="Resultados numéricos de la evolución del ángulo del péndulo", 
                 xlabel = "t", ylabel="\\theta", label="Euler, h=1/480")
plot!(tt2,thetas2, label="Euler, h=1/960")

### 3.5- Ejercicio

- **¿Son consistentes los resultados (es decir, son tan parecidas que no se distinguen a simple vista)?** 

> **Respuesta**
>
>???

### 3.6- Ejercicio

- **Repetir los cálculos con longitudes de paso $h$ cada vez más pequeñas (obteniendo por tanto aproximaciones cada vez más precisas) hasta asegurarnos de que obtenemos resultados consistentes (más en concreto, hasta que las gráficas de $\theta$ con respecto de $t$ de las dos aproximaciones más precisas no se distinguen en la resolución de la pantalla del ordenador).**

- **Comentar el proceso de obtención de aproximaciones cada vez más precisas de los valores de los ángulos.**

> **Comentarios**
>
> ???

### 3.7- Ejercicio

 - **Utilizar los resultados finales para guardar en un fichero los datos de $\theta$ para $n=24001$ valores de $t$ uniformemente espaciados en el intervalo $[0,50]$. Realizar, haciendo uso de dicho fichero, la simulación animada del péndulo inmerso en aire. ¿Parece ahora realista dicha simulación?** 


In [ ]:
data2="./angulos_pendulo_2.txt"
f2=open(data2,"w");
writedlm(f2,thetas2)
close(f2);

Ejecutar el codigo JavaScripts: <a href="http://www.ehu.eus/ccwmuura/irakaskuntza/sinum/animaciones/pendulo/pendulo_desde_fichero.html">Fichero html</a>

- **¿Se obtiene ahora una simulación realista?**

> **Respuesta**
> 
> ???

<a href="#top">Back to the top</a>

### Valoración

_Incluir aquí los comentarios de valoración de la tarea a entregar (dificultad, interés, etc, incluidas, si se quiere, sugerencias de mejora del ejercicio), así como una estimación del tiempo dedicado al trabajo de la semana (desglosado en el tiempo de estudio de material teórico, tiempo de dedicación a la participación activa o pasiva en los foros, tiempo de implementación y experimentación con los problemas prácticos planteados, y tiempo de preparación del documento jupyter final)._